<a href="https://colab.research.google.com/github/froggeric/Scrap-SiriusXM/blob/main/SiriusXM_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 2: Install Necessary Libraries (Run this cell first in Colab)
!pip install requests beautifulsoup4 pandas lxml --quiet

In [3]:
# --- Imports ---
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import urllib.parse
import json
import os
try:
    # To use Colab Secrets for API Keys
    from google.colab import userdata
    SECRETS_AVAILABLE = True
except ImportError:
    SECRETS_AVAILABLE = False
    # print("Warning: 'google.colab.userdata' not available...") # Keep silent unless debugging

# ==============================================================================
# === USER CONFIGURATION AREA ==================================================
# ==============================================================================

# --- Input / Output Files ---
# USER ACTION: Create this CSV file and upload to Colab or specify path.
# Expected columns: 'Name' (optional), 'Role_Type' (e.g., "DJ/Host", "Role"), 'Associated_Channel' (optional)
INPUT_CSV_FILE = "targets.csv"
OUTPUT_CSV_FILE = "siriusxm_contacts_output.csv" # Output filename

# --- Scraping Behavior ---
REQUEST_DELAY = 2       # Seconds delay between direct web requests (be polite!)
REQUEST_TIMEOUT = 15    # Seconds before timeout
MAX_RETRIES = 2         # Number of retries after the initial attempt for network errors
RETRY_DELAY = 5         # Seconds to wait before retrying a failed request
SAVE_INTERVAL = 25      # Save progress to CSV every N records processed

# --- API Keys / CSE ID ---
# USER ACTION: Store these in Colab Secrets (Key icon in left sidebar)
# Recommended Secret Names: GOOGLE_API_KEY, GOOGLE_CSE_ID

# --- Web Request Headers ---
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9',
}

# --- Google API ---
GOOGLE_SEARCH_API_URL = "https://www.googleapis.com/customsearch/v1"

# --- CSS Selectors (USER MUST VERIFY/UPDATE THESE by inspecting live websites!) ---
# These are plausible examples based on common structures but WILL likely need changes.

# SiriusXM Channel Page Selectors (Example - Verify!)
SIRIUSXM_CHANNELS_URL = "https://www.siriusxm.com/channels"
SIRIUSXM_CHANNEL_LIST_SELECTOR = "a[href*='/channels/']" # Try finding links pointing to specific channels
SIRIUSXM_CHANNEL_NAME_SELECTOR = "h4, div[role='heading']" # Try common heading tags or divs marked as headings
SIRIUSXM_CHANNEL_URL_SELECTOR = None # Set to None if the LIST_SELECTOR is the 'a' tag itself.

# LinkedIn Public Profile Selectors (Example - Verify!)
LINKEDIN_TITLE_SELECTOR = "title" # Usually reliable

# Social Media Profile Selectors (Example - Verify!)
SOCIAL_BIO_SELECTOR_OG = "meta[property='og:description']" # Common, try first
SOCIAL_BIO_SELECTOR_META = "meta[name='description']"      # Fallback

# Follower counts - Static selectors usually FAIL. Requires JS/Selenium/API.
SOCIAL_FOLLOWER_SELECTOR = None # Explicitly None as requests/BS4 cannot reliably get this

# ==============================================================================
# === END USER CONFIGURATION AREA ==============================================
# ==============================================================================


# --- Helper Functions ---
def get_soup(url):
    """
    Fetches a URL and returns a BeautifulSoup object, with error handling and retries.
    Retries on Timeout, ConnectionError, and general RequestException.
    """
    soup = None
    for attempt in range(MAX_RETRIES + 1):
        try:
            print(f"Fetching: {url} (Attempt {attempt + 1}/{MAX_RETRIES + 1})")
            response = requests.get(url, headers=HEADERS, timeout=REQUEST_TIMEOUT)
            response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
            soup = BeautifulSoup(response.content, 'lxml')
            # print(f"Successfully fetched: {url}") # Reduce noise
            break # Exit loop on success

        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError, requests.exceptions.RequestException) as e:
            print(f"Warning: Network error fetching {url}: {e}")
            if attempt < MAX_RETRIES:
                print(f"Retrying in {RETRY_DELAY} seconds...")
                time.sleep(RETRY_DELAY)
                continue # <<<--- CORRECTED: Added continue to proceed to next attempt
            else:
                print(f"Error: Max retries reached for {url}. Giving up.")
                soup = None # Ensure None on final failure

        except requests.exceptions.HTTPError as e:
             print(f"Error: HTTP Error {e.response.status_code} fetching {url} (Not retrying).")
             soup = None # Ensure None on non-retryable HTTP error
             break # Exit loop

        except Exception as e:
            print(f"Error: Could not process {url}: {e} (Not retrying).")
            soup = None # Ensure None on other errors
            break # Exit loop

    # Delay slightly even if successful, more if failed? No, keep consistent delay after attempts.
    time.sleep(REQUEST_DELAY / 2) # Slightly shorter delay after direct fetches ok
    return soup

def clean_text(text):
    """Helper to strip whitespace and potentially clean up text."""
    return text.strip() if text else None


# --- Search Functions ---
def search_engine_lookup(query, api_key=None, cse_id=None, num_results=5):
    """
    Executes a web search using the Google Custom Search JSON API with retries.
    """
    # --- Get API Key and CSE ID securely ---
    if not api_key:
        if SECRETS_AVAILABLE:
            try: api_key = userdata.get('GOOGLE_API_KEY')
            except userdata.SecretNotFoundError: print("Error: Google API Key not found in Colab Secrets ('GOOGLE_API_KEY')."); return []
        else:
            api_key = os.environ.get('GOOGLE_API_KEY');
            if not api_key: print("Error: GOOGLE_API_KEY not found in environment variables."); return []
    if not cse_id:
        if SECRETS_AVAILABLE:
            try: cse_id = userdata.get('GOOGLE_CSE_ID')
            except userdata.SecretNotFoundError: print("Error: Google CSE ID not found in Colab Secrets ('GOOGLE_CSE_ID')."); return []
        else:
            cse_id = os.environ.get('GOOGLE_CSE_ID');
            if not cse_id: print("Error: GOOGLE_CSE_ID not found in environment variables."); return []

    params = {'key': api_key, 'cx': cse_id, 'q': query, 'num': num_results}
    results_found = []
    response_json = None

    for attempt in range(MAX_RETRIES + 1):
        try:
            print(f"Executing Google API search for: '{query}' (Attempt {attempt + 1}/{MAX_RETRIES + 1})")
            response = requests.get(GOOGLE_SEARCH_API_URL, params=params, headers=HEADERS, timeout=REQUEST_TIMEOUT)
            response.raise_for_status()
            response_json = response.json() # Try decoding JSON here
            # print(f"Successfully got API response for: '{query}'") # Reduce noise
            break # Exit loop on success

        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError, requests.exceptions.RequestException) as e:
            print(f"Warning: Network error during Google API search for '{query}': {e}")
            if attempt < MAX_RETRIES:
                print(f"Retrying in {RETRY_DELAY} seconds...")
                time.sleep(RETRY_DELAY)
                continue # <<<--- CORRECTED: Added continue to proceed to next attempt
            else:
                print(f"Error: Max retries reached for Google API search '{query}'. Giving up.")
                return [] # Return empty on final failure

        except requests.exceptions.HTTPError as e:
             print(f"Error: HTTP Error {e.response.status_code} during Google API search for '{query}' (Not retrying). Status: {e.response.text}")
             return []

        except json.JSONDecodeError as e:
             print(f"Error: Could not decode JSON response from Google API for '{query}' (Not retrying). Response text: {response.text[:200]}...")
             return []

        except Exception as e:
            print(f"Error: An unexpected error occurred during Google API search for '{query}': {e} (Not retrying).")
            return []

    # --- Parse Response ---
    if response_json:
        if 'error' in response_json:
            error_details = response_json['error'].get('message', 'Unknown API error')
            print(f"Error: Google API returned an application-level error: {error_details}")
            return []
        items = response_json.get('items', [])
        # print(f"No search results found via API for '{query}'.") # Only print if needed
        for item in items:
            link = item.get('link'); title = item.get('title'); snippet = item.get('snippet')
            if link: results_found.append({'link': link, 'title': title, 'snippet': snippet})
        print(f"Found {len(results_found)} results via Google API for '{query}'.")
    else:
        print(f"No valid response received after retries for Google API search '{query}'.")
        return []

    time.sleep(max(REQUEST_DELAY / 4, 0.5)) # Short delay after successful API call
    return results_found

def search_reddit(query, api_key=None, cse_id=None, num_results=3):
    """Searches Reddit using the Google Custom Search API by adding 'site:reddit.com'."""
    reddit_query = f"site:reddit.com {query}"
    reddit_results = search_engine_lookup(query=reddit_query, api_key=api_key, cse_id=cse_id, num_results=num_results)
    return reddit_results


# --- Scraping Functions ---
def scrape_siriusxm_channels(url):
    """Scrapes the main SiriusXM channels page for channel names and URLs."""
    print("Scraping SiriusXM Channel List...")
    soup = get_soup(url) # Uses retry logic
    channels_data = []
    if not soup: print("Failed to get SiriusXM channel page soup after retries."); return channels_data

    # --- Using EXAMPLE selector - USER MUST VERIFY/UPDATE ---
    channel_link_elements = soup.select(SIRIUSXM_CHANNEL_LIST_SELECTOR)
    if not channel_link_elements: print(f"Warning: SiriusXM channel selector '{SIRIUSXM_CHANNEL_LIST_SELECTOR}' not found."); return channels_data

    processed_urls = set()
    for element in channel_link_elements:
        try:
            channel_url_relative = element.get('href')
            if not channel_url_relative or not channel_url_relative.startswith('/channels/'): continue
            channel_url_absolute = urllib.parse.urljoin(url, channel_url_relative)
            if channel_url_absolute in processed_urls: continue
            processed_urls.add(channel_url_absolute)

            # --- Using EXAMPLE selectors - USER MUST VERIFY/UPDATE ---
            name_tag = element.select_one(SIRIUSXM_CHANNEL_NAME_SELECTOR)
            name = clean_text(name_tag.get_text()) if name_tag else None
            if not name: name = clean_text(element.get('aria-label'))
            if not name:
                img_tag = element.find('img');
                if img_tag: name = clean_text(img_tag.get('alt'))
            if not name and channel_url_relative:
                 name_guess = channel_url_relative.split('/')[-1]
                 if name_guess: name = name_guess.replace('-', ' ').title()

            if name: channels_data.append({"channel_name": name, "channel_url": channel_url_absolute})
            else: print(f"Warning: Could not determine name for channel URL: {channel_url_absolute}")
        except Exception as e: print(f"Error parsing a channel element: {e}")

    print(f"Found {len(channels_data)} potential channels using example selectors.")
    return channels_data

def scrape_profile_metadata(profile_url):
    """Attempts to scrape basic PUBLIC metadata from a profile URL (LinkedIn, Social)."""
    metadata = {"title": None, "bio_snippet": None, "url": profile_url}
    if not profile_url: return metadata
    soup = get_soup(profile_url) # Uses retry logic
    if not soup: return metadata # Return empty if fetch failed after retries

    try:
        # --- Using EXAMPLE selector - USER MUST VERIFY/UPDATE ---
        title_tag = soup.select_one(LINKEDIN_TITLE_SELECTOR)
        if title_tag: metadata["title"] = clean_text(title_tag.get_text())

        # --- Using EXAMPLE selectors - Try multiple patterns for Bio ---
        bio = None
        og_bio_tag = soup.select_one(SOCIAL_BIO_SELECTOR_OG)
        if og_bio_tag and og_bio_tag.has_attr('content'): bio = clean_text(og_bio_tag['content'])
        if not bio:
            meta_bio_tag = soup.select_one(SOCIAL_BIO_SELECTOR_META)
            if meta_bio_tag and meta_bio_tag.has_attr('content'): bio = clean_text(meta_bio_tag['content'])
        metadata["bio_snippet"] = bio

    except Exception as e: print(f"Error parsing metadata from {profile_url}: {e}")
    return metadata


# --- Reddit Parsing Function ---
def parse_reddit_findings(reddit_results):
    """Analyzes Reddit results for common themes/strategies."""
    notes = []
    has_linkedin_mention = False; has_official_address_mention = False
    for result in reddit_results:
        content = (result.get("title", "") + " " + result.get("snippet", "")).lower()
        if "linkedin" in content and ("program director" in content or "music director" in content or "contact" in content):
            if not has_linkedin_mention: notes.append("Reddit users recommend using LinkedIn to find PDs/MDs."); has_linkedin_mention = True
        if "1221 avenue of the americas" in content or "official mail" in content:
             if not has_official_address_mention: notes.append("Reddit confirms official NYC mail-in address."); has_official_address_mention = True
        if "email format" in content or re.search(r'\b[a-z0-9._%+-]+@[a-z0-9.-]+\.com\b', content): # Broadened regex slightly
            # Only add note once, avoid specific speculative formats
             if "speculation" not in " ".join(notes): notes.append("Reddit may contain *speculation* on email formats (treat with extreme caution).")
    return " ".join(notes) if notes else "No specific strategy insights found in Reddit search results."


# --- Main Execution Logic ---
def main():
    """Main function to orchestrate the scraping and data export."""
    all_data = []; processed_linkedin_urls = set(); processed_count = 0

    # --- Load Targets ---
    try:
        print(f"Loading targets from {INPUT_CSV_FILE}...")
        targets_df = pd.read_csv(INPUT_CSV_FILE); targets = targets_df.to_dict('records')
        print(f"Loaded {len(targets)} targets.")
    except FileNotFoundError: print(f"Error: Input file '{INPUT_CSV_FILE}' not found."); return
    except Exception as e: print(f"Error reading input file {INPUT_CSV_FILE}: {e}"); return

    print("\n--- Starting Processing ---")
    # --- Process Targets Loop ---
    for target in targets:
        target_name = target.get("Name"); target_role_type = target.get("Role_Type"); target_channel = target.get("Associated_Channel")
        if not target_role_type: print(f"Skipping row with missing 'Role_Type': {target}"); continue
        print(f"\nProcessing Target: Name='{target_name}', Role='{target_role_type}', Channel='{target_channel}'")

        # --- Construct Search Query ---
        query_parts = []
        if target_name: query_parts.append(f'"{target_name}"')
        # If Role_Type is 'Role', the Name column likely holds the actual role title
        if target_role_type == "Role" and target_name: query_parts = [f'"{target_name}"']
        # Don't add Role_Type if it was already used as the Name
        elif target_role_type and target_role_type != "Role": query_parts.append(f'"{target_role_type}"')
        query_parts.append("SiriusXM")
        if target_channel: query_parts.append(f'"{target_channel}"')
        search_query = " ".join(query_parts)

        # --- Search Engine Lookup ---
        found_results = search_engine_lookup(search_query)

        # --- Process Found URLs ---
        linkedin_url = None; twitter_url = None; instagram_url = None; other_urls = []
        for result_item in found_results:
            url = result_item.get('link')
            if not url: continue # Skip if no URL in this result item

            # Assign URLs (take first found for each type)
            if ("linkedin.com/in/" in url or "linkedin.com/pub/" in url) and not linkedin_url: linkedin_url = url
            elif "twitter.com/" in url and not twitter_url: twitter_url = url
            elif "instagram.com/" in url and not instagram_url: instagram_url = url
            elif any(domain in url for domain in ["facebook.com/", "tiktok.com/", "siriusxm.com/hosts/"]): other_urls.append(url)

        # --- Deduplication Check ---
        if linkedin_url and linkedin_url in processed_linkedin_urls:
            print(f"Skipping duplicate LinkedIn profile: {linkedin_url}")
            continue

        # --- Prepare Data Entry ---
        entry = {
            "Name": target_name, "Role_Type": target_role_type, "Associated_Channel": target_channel,
            "LinkedIn_URL": linkedin_url, "LinkedIn_Title": None, "Twitter_URL": twitter_url,
            "Instagram_URL": instagram_url, "Other_Social_URL": "; ".join(other_urls) if other_urls else None,
            "Social_Bio_Snippet": None, "Reddit_Insights": None, "Notes": "Requires manual verification."
        }

        # --- Metadata Scraping ---
        if linkedin_url:
            linkedin_meta = scrape_profile_metadata(linkedin_url)
            entry["LinkedIn_Title"] = linkedin_meta.get("title")
            processed_linkedin_urls.add(linkedin_url) # Add only if scrape attempted
            # Refine name based on LinkedIn title if input name was blank or generic role title
            if (not entry["Name"] or entry["Name"].lower() in ["program director", "music director", "host", "dj", "curator"]) and entry["LinkedIn_Title"]:
                 title_parts = re.split(r'\s*-\s*|\s*\|\s*', entry["LinkedIn_Title"], 1)
                 if title_parts and title_parts[0].strip(): entry["Name"] = title_parts[0].strip() # Update name

        social_bio = None
        if twitter_url:
             twitter_meta = scrape_profile_metadata(twitter_url); social_bio = twitter_meta.get("bio_snippet")
        if instagram_url and not social_bio:
             ig_meta = scrape_profile_metadata(instagram_url); social_bio = ig_meta.get("bio_snippet")
        entry["Social_Bio_Snippet"] = social_bio

        # --- Reddit Context ---
        reddit_search_query_parts = ["SiriusXM"]
        if entry["Name"]: reddit_search_query_parts.append(f'"{entry["Name"]}"') # Use potentially updated name
        if target_channel: reddit_search_query_parts.append(f'"{target_channel}"')
        reddit_search_query_parts.append("contact OR email OR submit")
        reddit_search_results = search_reddit(" ".join(reddit_search_query_parts))
        entry["Reddit_Insights"] = parse_reddit_findings(reddit_search_results)

        # --- Store Entry & Update Count ---
        all_data.append(entry)
        processed_count += 1
        print(f"Processed entry #{processed_count} for '{entry['Name']}'")

        # --- Periodic Saving ---
        if processed_count > 0 and processed_count % SAVE_INTERVAL == 0:
            print(f"\n--- Saving progress ({processed_count} records processed) to {OUTPUT_CSV_FILE} ---")
            try:
                # Create DataFrame from current data
                temp_df = pd.DataFrame(all_data)
                # Define columns explicitly for consistent saving
                final_columns = ["Name", "Role_Type", "Associated_Channel", "LinkedIn_URL", "LinkedIn_Title", "Twitter_URL", "Instagram_URL", "Other_Social_URL", "Social_Bio_Snippet", "Reddit_Insights", "Notes"]
                # Add any missing columns with None/NaN before saving
                for col in final_columns:
                    if col not in temp_df.columns: temp_df[col] = None
                temp_df = temp_df[final_columns] # Reorder
                temp_df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8')
                print("--- Progress saved successfully. ---")
            except Exception as e: print(f"--- Error saving progress: {e} ---")
            print("-" * 20) # Separator

    # --- Final Export ---
    print("\n--- Processing Complete ---")
    if not all_data: print("No data collected."); return

    df = pd.DataFrame(all_data)
    final_columns = ["Name", "Role_Type", "Associated_Channel", "LinkedIn_URL", "LinkedIn_Title", "Twitter_URL", "Instagram_URL", "Other_Social_URL", "Social_Bio_Snippet", "Reddit_Insights", "Notes"]
    for col in final_columns:
        if col not in df.columns: df[col] = None
    df = df[final_columns]
    try:
        df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8')
        print(f"Final data exported successfully to {OUTPUT_CSV_FILE} ({len(df)} records)")
    except Exception as e: print(f"Error exporting final data to CSV: {e}")

# --- Run the Script ---
if __name__ == "__main__":
    main()

SyntaxError: invalid syntax (<ipython-input-3-ae1a3033a841>, line 288)